# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

***
*WORK IN PROGRESS*
***

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hopspitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hopspitalization, ICU stats --PENDING--

### --PENDING-- US County - Alternative: NY Times
- Link: https://github.com/nytimes/covid-19-data/
- **Historic time series data:**
    - US States (cases, deaths) --PENDING--
    - US Counties (cases, deaths) --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-13-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-13-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-06-14 03:33:15,34.223334,-82.461707,63,0,0,63,"Abbeville, South Carolina, US",256.859787,0.000000
1,22001.0,Acadia,Louisiana,US,2020-06-14 03:33:15,30.295065,-92.414197,577,31,0,546,"Acadia, Louisiana, US",929.970183,5.372617
2,51001.0,Accomack,Virginia,US,2020-06-14 03:33:15,37.767072,-75.632346,989,14,0,975,"Accomack, Virginia, US",3060.403515,1.415571
3,16001.0,Ada,Idaho,US,2020-06-14 03:33:15,43.452658,-116.241552,877,22,0,855,"Ada, Idaho, US",182.106245,2.508552
4,19001.0,Adair,Iowa,US,2020-06-14 03:33:15,41.330756,-94.471059,11,0,0,11,"Adair, Iowa, US",153.803132,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until day is over.**

In [6]:
jh_cases_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_cases_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,18054,18969,19551,20342,20917,21459,22142,22890,23546,24102
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1197,1212,1232,1246,1263,1299,1341,1385,1416,1464
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,9831,9935,10050,10154,10265,10382,10484,10589,10698,10810


In [7]:
jh_deaths_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_deaths_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,300,309,327,357,369,384,405,426,446,451
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,33,33,34,34,34,34,34,35,36,36
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,681,690,698,707,715,724,732,741,751,760


In [8]:
jh_recovered_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_recovered_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1585,1762,1830,1875,2171,2651,3013,3326,3928,4201
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,898,910,925,938,945,960,980,1001,1034,1039
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,6297,6453,6631,6717,6799,6951,7074,7255,7322,7420


In [9]:
jh_cases_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_cases_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,179,179,179,179,179,180,180,183,183,183
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,26,26,26,27,28,30,30,30,30,30


In [10]:
jh_deaths_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_deaths_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [11]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
451,1019.0,Cherokee,Alabama,US,2020-06-14 03:33:15,34.178060,-85.606390,47,5,0,42,"Cherokee, Alabama, US",179.416705,10.638298
913,1059.0,Franklin,Alabama,US,2020-06-14 03:33:15,34.442353,-87.842895,709,10,0,699,"Franklin, Alabama, US",2260.697660,1.410437
116,1003.0,Baldwin,Alabama,US,2020-06-14 03:33:15,30.727750,-87.722071,361,9,0,352,"Baldwin, Alabama, US",161.713717,2.493075


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [12]:
jh_countries_live = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [13]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,2074526,115436,556606
Brazil,850514,42720,459436
Russia,519458,6819,274128


In [14]:
jh_countries_live['Active'] = jh_countries_live['Confirmed'] - jh_countries_live['Deaths'] - jh_countries_live['Recovered']

In [15]:
jh_countries_live['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_countries_live['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_countries_live['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [16]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,2074526,115436,556606,1402484,2020-06-14 03:33:15,38.251306,-91.424911
Brazil,850514,42720,459436,348358,2020-06-14 03:33:15,-12.669522,-48.480493
Russia,519458,6819,274128,238511,2020-06-14 03:33:15,54.546312,62.120860
India,308993,8884,154330,145779,2020-06-14 03:33:15,23.356511,82.081064
United Kingdom,295828,41747,1283,252798,2020-06-14 03:33:15,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [17]:
jh_states_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [18]:
jh_states_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,382630,30795,0
New Jersey,166605,12621,0
California,150018,5076,0


In [19]:
jh_states_live['Active'] = jh_states_live['Confirmed'] - jh_states_live['Deaths'] - jh_states_live['Recovered']

In [20]:
jh_states_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_states_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_states_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [21]:
jh_states_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,382630,30795,0,351835,2020-06-14 03:33:15,42.671593,-75.579694
New Jersey,166605,12621,0,153984,2020-06-14 03:33:15,40.286441,-74.622450
California,150018,5076,0,144942,2020-06-14 03:33:15,37.778248,-120.728666
Illinois,131871,6289,0,125582,2020-06-14 03:33:15,39.845605,-89.165461
Massachusetts,105395,7576,0,97819,2020-06-14 03:33:15,42.176400,-71.577050


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [22]:
jh_counties_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

Confirmed  Deaths  Recovered
Province_State Admin2                                     
New York       New York City     209493   22076          0
Illinois       Cook               84581    4173          0
California     Los Angeles        72057    2894          0
New York       Nassau             41172    2163          0
               Suffolk            40615    1947          0

In [24]:
jh_counties_live['Active'] = jh_counties_live['Confirmed'] - jh_counties_live['Deaths'] - jh_counties_live['Recovered']


In [25]:
jh_counties_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_counties_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_counties_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_counties_live['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [26]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

Confirmed  Deaths  Recovered  Active  \
Province_State Admin2                                                
New York       New York City     209493   22076          0  187417   
Illinois       Cook               84581    4173          0   80408   
California     Los Angeles        72057    2894          0   69163   
New York       Nassau             41172    2163          0   39009   
               Suffolk            40615    1947          0   38668   

                                      Last_Update        Lat        Long  \
Province_State Admin2                                                      
New York       New York City  2020-06-14 03:33:15  40.767273  -73.971526   
Illinois       Cook           2020-06-14 03:33:15  41.841448  -87.816588   
California     Los Angeles    2020-06-14 03:33:15  34.308284 -118.228241   
New York       Nassau         2020-06-14 03:33:15  40.740665  -73.589419   
               Suffolk        2020-06-14 03:33:15  40.883201  -72.801217   

                                 FIPS  
Province_State Admin2                  
New York       New York City  36061.0  
Illinois       Cook           17031.0  
California     Los Angeles     6037.0  
New York       Nassau         36059.0  
               Suffolk        36103.0

##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [27]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_cases_global_hist.iloc[:, 4:].columns

In [28]:
jh_cases_countries_hist_ts = pd.pivot_table(jh_cases_global_hist, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.transpose()
jh_cases_countries_hist_ts.index = pd.to_datetime(jh_cases_countries_hist_ts.index)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.sort_index()

In [29]:
jh_cases_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-11,22890,1385,10589,852,118,26,27373,14669,7289,17034,...,292860,847,4741,2814,332,487,9,591,1200,332
2020-06-12,23546,1416,10698,853,130,26,28764,15281,7294,17064,...,294402,847,4869,2879,333,489,9,632,1321,343
2020-06-13,24102,1464,10810,853,138,26,30295,16004,7320,17078,...,295828,847,4966,2904,334,489,9,705,1357,356


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [30]:
jh_deaths_countries_hist_ts = pd.pivot_table(jh_deaths_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.transpose()
jh_deaths_countries_hist_ts.index = pd.to_datetime(jh_deaths_countries_hist_ts.index)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.sort_index()

In [31]:
jh_deaths_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-11,426,35,741,51,5,3,765,245,102,674,...,41364,23,19,23,0,3,1,136,10,4
2020-06-12,446,36,751,51,5,3,785,258,102,675,...,41566,23,19,23,0,3,1,139,10,4
2020-06-13,451,36,760,51,6,3,815,264,102,677,...,41747,23,19,24,0,3,1,160,10,4


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [32]:
jh_recovered_countries_hist_ts = pd.pivot_table(jh_recovered_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.transpose()
jh_recovered_countries_hist_ts.index = pd.to_datetime(jh_recovered_countries_hist_ts.index)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.sort_index()

In [33]:
jh_recovered_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-11,3326,1001,7255,780,41,20,8332,5466,6782,15949,...,1278,772,3637,487,321,410,6,23,912,51
2020-06-12,3928,1034,7322,781,42,20,8743,5639,6803,15985,...,1282,780,3758,487,323,414,6,28,1104,51
2020-06-13,4201,1039,7420,781,61,20,9083,6081,6838,16012,...,1283,784,3874,487,323,415,8,39,1104,54


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [34]:
jh_cases_counties_hist.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,708,714,730,739,744,752,764,786,811,849
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,124,125,126,126,126,126,126,129,132,134
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,175,179,182,184,187,187,192,200,204,206
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,93,94,96,98,100,104,105,110,117,122


In [35]:
date_columns = jh_cases_counties_hist.iloc[:,11:].columns

In [36]:
jh_cases_states_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.transpose()
jh_cases_states_hist_ts.index = pd.to_datetime(jh_cases_states_hist_ts.index)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.sort_index()

In [37]:
jh_cases_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-11,22845,609,0,31267,10816,143377,28632,44461,10106,49,...,28340,82658,13252,1110,72,52647,24779,2217,21926,1009
2020-06-12,23710,624,0,33039,11547,146659,28807,44689,10173,49,...,29118,84927,13577,1119,72,53211,25171,2249,22246,1027
2020-06-13,24601,653,0,34660,12095,150018,29002,44994,10229,49,...,29549,86915,13981,1125,72,53869,25538,2274,22518,1050


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [38]:
jh_cases_counties_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.transpose()
jh_cases_counties_hist_ts.index = pd.to_datetime(jh_cases_counties_hist_ts.index)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.sort_index()

In [39]:
jh_cases_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-11         312     343     214   93     87     253    484     184   
2020-06-12         323     353     221   97     95     258    499     188   
2020-06-13         331     361     226  100    102     276    517     190   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-11          417       42  ...       2      1       19        3   
2020-06-12          427       46  ...       3      1       19        3   
2020-06-13          438       47  ...       5      1       19        3   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-11             36   102    53          0       39      1  
2020-06-12             38   104    62          0       39      1  
2020-06-13             40   104    71          0       39      1  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [40]:
jh_deaths_states_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.transpose()
jh_deaths_states_hist_ts.index = pd.to_datetime(jh_deaths_states_hist_ts.index)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.sort_index()

In [41]:
jh_deaths_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-11,755,11,0,1135,171,4934,1582,4146,414,0,...,441,1930,131,55,6,1520,1194,86,682,18
2020-06-12,769,12,0,1156,176,4978,1582,4159,414,0,...,466,1956,139,55,6,1534,1204,88,689,18
2020-06-13,773,12,0,1189,177,5076,1597,4186,419,0,...,472,1971,139,55,6,1541,1213,88,691,18


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [42]:
jh_deaths_counties_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.transpose()
jh_deaths_counties_hist_ts.index = pd.to_datetime(jh_deaths_counties_hist_ts.index)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.sort_index()

In [43]:
jh_deaths_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-11           6       9       1    1      1       8     25       3   
2020-06-12           6       9       1    1      1       8     25       3   
2020-06-13           6       9       1    1      1       8     25       3   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-11           26        4  ...       0      0        0        0   
2020-06-12           26        5  ...       0      0        0        0   
2020-06-13           26        5  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-11              0     0     0         17        0      0  
2020-06-12              0     0     0         17        0      0  
2020-06-13              0     0     0         17        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [44]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_countries_live.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [45]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_states_live.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [46]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_counties_live.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [47]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_cases_countries_hist_ts.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [48]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_deaths_countries_hist_ts.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_recovered_countries_hist_ts.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_cases_states_hist_ts.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_cases_counties_hist_ts.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_deaths_states_hist_ts.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_deaths_counties_hist_ts.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [54]:
jh_cases_countries_hist_pct = jh_cases_countries_hist_ts.pct_change()

In [55]:
jh_cases_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-11,0.033782,0.032811,0.010015,0.000000,0.044248,0.0,0.053334,0.040133,0.000549,0.001705,...,0.004362,0.0,0.025525,0.027757,0.000000,0.004124,0.0,0.055357,0.000000,0.037500
2020-06-12,0.028659,0.022383,0.010294,0.001174,0.101695,0.0,0.050816,0.041721,0.000686,0.001761,...,0.005265,0.0,0.026999,0.023099,0.003012,0.004107,0.0,0.069374,0.100833,0.033133
2020-06-13,0.023613,0.033898,0.010469,0.000000,0.061538,0.0,0.053226,0.047314,0.003565,0.000820,...,0.004844,0.0,0.019922,0.008684,0.003003,0.000000,0.0,0.115506,0.027252,0.037901


In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_cases_countries_hist_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [57]:
jh_deaths_countries_hist_pct = jh_deaths_countries_hist_ts.pct_change()

In [58]:
jh_deaths_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-11,0.051852,0.029412,0.012295,0.0,0.25,0.0,0.040816,0.079295,0.0,0.001486,...,0.003664,0.0,0.0,0.000000,NaN,0.0,0.0,0.054264,0.0,0.0
2020-06-12,0.046948,0.028571,0.013495,0.0,0.00,0.0,0.026144,0.053061,0.0,0.001484,...,0.004883,0.0,0.0,0.000000,NaN,0.0,0.0,0.022059,0.0,0.0
2020-06-13,0.011211,0.000000,0.011984,0.0,0.20,0.0,0.038217,0.023256,0.0,0.002963,...,0.004355,0.0,0.0,0.043478,NaN,0.0,0.0,0.151079,0.0,0.0


In [59]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_deaths_countries_hist_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [60]:
jh_recovered_countries_hist_pct = jh_recovered_countries_hist_ts.pct_change()

In [61]:
jh_recovered_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-11,0.103883,0.021429,0.025587,0.027668,0.025000,0.0,0.042673,0.045924,0.003403,0.002451,...,0.007092,0.018470,0.029728,0.0,0.003125,0.000000,0.000000,0.000000,0.000000,0.040816
2020-06-12,0.180998,0.032967,0.009235,0.001282,0.024390,0.0,0.049328,0.031650,0.003096,0.002257,...,0.003130,0.010363,0.033269,0.0,0.006231,0.009756,0.000000,0.217391,0.210526,0.000000
2020-06-13,0.069501,0.004836,0.013384,0.000000,0.452381,0.0,0.038888,0.078383,0.005145,0.001689,...,0.000780,0.005128,0.030867,0.0,0.000000,0.002415,0.333333,0.392857,0.000000,0.058824


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_recovered_countries_hist_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [63]:
jh_cases_states_hist_pct = jh_cases_states_hist_ts.pct_change()

In [64]:
jh_cases_states_hist_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-11,0.038929,0.028716,NaN,0.047401,0.043210,0.026210,0.005196,0.002571,0.004972,0.0,...,0.014680,0.023286,0.030162,0.013699,0.0,0.009008,0.017451,0.010483,0.015422,0.029592
2020-06-12,0.037864,0.024631,NaN,0.056673,0.067585,0.022891,0.006112,0.005128,0.006630,0.0,...,0.027452,0.027450,0.024525,0.008108,0.0,0.010713,0.015820,0.014434,0.014595,0.017839
2020-06-13,0.037579,0.046474,NaN,0.049063,0.047458,0.022903,0.006769,0.006825,0.005505,0.0,...,0.014802,0.023408,0.029756,0.005362,0.0,0.012366,0.014580,0.011116,0.012227,0.022395


In [65]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_cases_states_hist_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [66]:
jh_cases_counties_hist_pct = jh_cases_counties_hist_ts.pct_change()

In [67]:
jh_cases_counties_hist_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-11      0.057627  0.036254  0.028846  0.044944  0.101266  0.020161   
2020-06-12      0.035256  0.029155  0.032710  0.043011  0.091954  0.019763   
2020-06-13      0.024768  0.022663  0.022624  0.030928  0.073684  0.069767   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-06-11      0.027601  0.010989  0.039900  0.000000  ...  0.000000    0.0   
2020-06-12      0.030992  0.021739  0.023981  0.095238  ...  0.500000    0.0   
2020-06-13      0.036072  0.010638  0.025761  0.021739  ...  0.666667    0.0   

Province_State                                                              \
Admin2         Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-06-11          0.0      0.0   0.028571  0.009901  0.358974        NaN   
2020-06-12          0.0      0.0   0.055556  0.019608  0.169811        NaN   
2020-06-13          0.0      0.0   0.052632  0.000000  0.145161        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-06-11          0.0    0.0  
2020-06-12          0.0    0.0  
2020-06-13          0.0    0.0  

[3 rows x 3254 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_cases_counties_hist_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_deaths_states_hist_pct = jh_deaths_states_hist_ts.pct_change()

In [70]:
jh_deaths_states_hist_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-11,0.014785,0.000000,NaN,0.030881,0.036364,0.016481,0.006361,0.006311,0.002421,NaN,...,0.009153,0.013123,0.023438,0.0,0.0,0.003963,0.015306,0.011765,0.016393,0.0
2020-06-12,0.018543,0.090909,NaN,0.018502,0.029240,0.008918,0.000000,0.003136,0.000000,NaN,...,0.056689,0.013472,0.061069,0.0,0.0,0.009211,0.008375,0.023256,0.010264,0.0
2020-06-13,0.005202,0.000000,NaN,0.028547,0.005682,0.019687,0.009482,0.006492,0.012077,NaN,...,0.012876,0.007669,0.000000,0.0,0.0,0.004563,0.007475,0.000000,0.002903,0.0


In [71]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_deaths_states_hist_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [72]:
jh_deaths_counties_hist_pct = jh_deaths_counties_hist_ts.pct_change()

In [73]:
jh_deaths_counties_hist_pct.tail(3)

Province_State Alabama                                                        \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock    Butler Calhoun   
2020-06-11         0.0     0.0     0.0  0.0    0.0     0.0  0.041667     0.0   
2020-06-12         0.0     0.0     0.0  0.0    0.0     0.0  0.000000     0.0   
2020-06-13         0.0     0.0     0.0  0.0    0.0     0.0  0.000000     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-11          0.0     0.00  ...     NaN    NaN      NaN      NaN   
2020-06-12          0.0     0.25  ...     NaN    NaN      NaN      NaN   
2020-06-13          0.0     0.00  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-11            NaN   NaN   NaN        0.0      NaN    NaN  
2020-06-12            NaN   NaN   NaN        0.0      NaN    NaN  
2020-06-13            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3254 columns]

In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_deaths_counties_hist_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [107]:
jh_cases_countries_hist_pct_ewma = jh_cases_countries_hist_pct.ewm(span=3).mean()

In [108]:
jh_cases_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-11,0.032351,0.029721,0.010245,0.000122,0.073963,0.000010,0.049881,0.035620,0.000846,0.001649,...,0.004394,0.000697,0.024744,0.037370,0.000427,0.007019,9.536743e-07,0.054912,0.006388,0.039029
2020-06-12,0.030505,0.026052,0.010269,0.000648,0.087829,0.000005,0.050349,0.038670,0.000766,0.001705,...,0.004829,0.000348,0.025871,0.030234,0.001719,0.005563,4.768372e-07,0.062143,0.053610,0.036081
2020-06-13,0.027059,0.029975,0.010369,0.000324,0.074684,0.000002,0.051787,0.042992,0.002165,0.001263,...,0.004837,0.000174,0.022897,0.019459,0.002361,0.002782,2.384186e-07,0.088825,0.040431,0.036991


In [109]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_cases_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [ ]:
jh_deaths_countries_hist_pct_ewma = jh_deaths_countries_hist_pct.ewm(span=3).mean()

In [ ]:
jh_deaths_countries_hist_pct_ewma.tail(3)

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_deaths_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [111]:
jh_recovered_countries_hist_pct_ewma = jh_recovered_countries_hist_pct.ewm(span=3).mean()

In [112]:
jh_recovered_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-11,0.126152,0.019216,0.021959,0.016315,0.065651,0.000051,0.047651,0.082089,0.002990,0.002332,...,0.007272,0.014460,0.025517,0.010769,0.006757,0.005111,4.656613e-11,0.003132,0.000104,0.060501
2020-06-12,0.153575,0.026092,0.015597,0.008799,0.045020,0.000026,0.048489,0.056870,0.003043,0.002294,...,0.005201,0.012412,0.029393,0.005385,0.006494,0.007434,2.328306e-11,0.110262,0.105315,0.030250
2020-06-13,0.111538,0.015464,0.014491,0.004399,0.248701,0.000013,0.043689,0.067626,0.004094,0.001992,...,0.002990,0.008770,0.030130,0.002692,0.003247,0.004925,1.666667e-01,0.251559,0.052657,0.044537


In [113]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_recovered_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [114]:
jh_cases_states_hist_pct_ewma = jh_cases_states_hist_pct.ewm(span=3).mean()

In [115]:
jh_cases_states_hist_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-11,0.031811,0.028705,NaN,0.045170,0.037646,0.024147,0.005522,0.002967,0.004657,2.199948e-27,...,0.015419,0.024797,0.026885,0.012096,0.003584,0.009863,0.012046,0.009479,0.014240,0.020044
2020-06-12,0.034837,0.026668,NaN,0.050922,0.052615,0.023519,0.005817,0.004048,0.005644,1.099974e-27,...,0.021435,0.026124,0.025705,0.010102,0.001792,0.010288,0.013933,0.011956,0.014417,0.018942
2020-06-13,0.036208,0.036571,NaN,0.049992,0.050037,0.023211,0.006293,0.005436,0.005574,5.499871e-28,...,0.018119,0.024766,0.027731,0.007732,0.000896,0.011327,0.014257,0.011536,0.013322,0.020669


In [116]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_cases_states_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [117]:
jh_cases_counties_hist_pct_ewma = jh_cases_counties_hist_pct.ewm(span=3).mean()

In [118]:
jh_cases_counties_hist_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-11      0.048327  0.027544  0.029743  0.045580  0.070760  0.021032   
2020-06-12      0.041792  0.028349  0.031227  0.044295  0.081357  0.020398   
2020-06-13      0.033280  0.025506  0.026926  0.037612  0.077521  0.045083   

Province_State                                          ...       Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...          Park   
2020-06-11      0.022009  0.011379  0.030720  0.002406  ...  1.192093e-07   
2020-06-12      0.026500  0.016559  0.027350  0.048822  ...  2.500001e-01   
2020-06-13      0.031286  0.013599  0.026556  0.035281  ...  4.583334e-01   

Province_State                                                             \
Admin2                Platte  Sheridan      Sublette Sweetwater     Teton   
2020-06-11     -2.710507e-20  0.017795  4.440892e-16   0.027604  0.007450   
2020-06-12     -1.355253e-20  0.008898  2.220446e-16   0.041580  0.013529   
2020-06-13     -6.776264e-21  0.004449  1.110223e-16   0.047106  0.006765   

Province_State                                        
Admin2             Uinta Unassigned  Washakie Weston  
2020-06-11      0.273759        NaN  0.000228    0.0  
2020-06-12      0.221785        NaN  0.000114    0.0  
2020-06-13      0.183473        NaN  0.000057    0.0  

[3 rows x 3254 columns]

In [119]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_cases_counties_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [120]:
jh_deaths_states_hist_pct_ewma = jh_deaths_states_hist_pct.ewm(span=3).mean()

In [121]:
jh_deaths_states_hist_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-11,0.017544,0.012500,NaN,0.025080,0.030257,0.017400,0.007888,0.005440,0.007720,NaN,...,0.011103,0.013781,0.018670,5.654218e-07,1.513423e-10,0.007266,0.009752,0.009437,0.015389,0.014749
2020-06-12,0.018043,0.051705,NaN,0.021791,0.029748,0.013159,0.003944,0.004288,0.003860,NaN,...,0.033896,0.013626,0.039870,2.827109e-07,7.567116e-11,0.008238,0.009064,0.016347,0.012827,0.007374
2020-06-13,0.011622,0.025852,NaN,0.025169,0.017715,0.016423,0.006713,0.005390,0.007969,NaN,...,0.023386,0.010647,0.019935,1.413554e-07,3.783558e-11,0.006401,0.008269,0.008173,0.007865,0.003687


In [122]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_deaths_states_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [123]:
jh_deaths_counties_hist_pct_ewma = jh_deaths_counties_hist_pct.ewm(span=3).mean()

In [124]:
jh_deaths_counties_hist_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga       Baldwin Barbour Bibb Blount   Bullock   
2020-06-11      0.050163  5.999590e-08     0.0  0.0    0.0  0.010417   
2020-06-12      0.025081  2.999795e-08     0.0  0.0    0.0  0.005209   
2020-06-13      0.012541  1.499897e-08     0.0  0.0    0.0  0.002604   

Province_State                                              ... Wyoming  \
Admin2            Butler       Calhoun  Chambers  Cherokee  ...    Park   
2020-06-11      0.028598  5.551115e-17  0.000044  0.005239  ...     NaN   
2020-06-12      0.014299  2.775558e-17  0.000022  0.127619  ...     NaN   
2020-06-13      0.007150  1.387779e-17  0.000011  0.063810  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-06-11        NaN      NaN      NaN        NaN   NaN   NaN   0.015671   
2020-06-12        NaN      NaN      NaN        NaN   NaN   NaN   0.007835   
2020-06-13        NaN      NaN      NaN        NaN   NaN   NaN   0.003918   

Province_State                  
Admin2         Washakie Weston  
2020-06-11          NaN    NaN  
2020-06-12          NaN    NaN  
2020-06-13          NaN    NaN  

[3 rows x 3254 columns]

In [125]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_deaths_counties_hist_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

# TODO - CONTINUE FROM HERE

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/states | [CSV](https://covidtracking.com/api/states.csv)  
States daily 4 pm ET - /api/states/daily | [CSV](http://covidtracking.com/api/states/daily.csv)  
States info - /api/states/info | [CSV](https://covidtracking.com/api/states/info.csv)  
US current - /api/us | [CSV](https://covidtracking.com/api/us.csv)  
US daily - /api/us/daily | [CSV](https://covidtracking.com/api/us/daily.csv)  
Counties- /api/counties | [CSV](https://covidtracking.com/api/counties.csv)  

In [75]:
states_curr = pd.read_csv('https://covidtracking.com/api/states.csv', index_col='state')

In [76]:
states_hist = pd.read_csv('http://covidtracking.com/api/states/daily.csv')

In [77]:
us_curr = pd.read_csv('https://covidtracking.com/api/us.csv')
us_curr

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,lastModified,total,totalTestResults,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash
0,20200613,56,2063249,20977055,1698,27902,224420,5878,9430,2726,...,2020-06-13T00:00:00Z,23042002,23040304,23040304,695,1139,497323,25719,523042,8b794af030ef4ec8ce1ea000efb9de729da57a7b


In [78]:
us_hist = pd.read_csv('https://covidtracking.com/api/us/daily.csv')

### Dataset 3. US Counties - New York Times
https://github.com/nytimes/covid-19-data/

In [79]:
counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [80]:
counties.date = pd.to_datetime(counties.date)

In [81]:
nytdate = counties.date.max()
nytdate

Timestamp('2020-06-12 00:00:00')

In [82]:
counties.set_index('date', inplace=True)
counties.head()

,county,state,fips,cases,deaths
date,,,,,
2020-01-21,Snohomish,Washington,53061.0,1,0
2020-01-22,Snohomish,Washington,53061.0,1,0
2020-01-23,Snohomish,Washington,53061.0,1,0
2020-01-24,Cook,Illinois,17031.0,1,0
2020-01-24,Snohomish,Washington,53061.0,1,0


## Data Wrangling

Get the data into a format where it's machine readable for analytics and forecasting

In [83]:
# Covid Tracking Project - US State data.  Look at first 5 rows before reformatting
states_hist.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200613,AK,654.0,71149.0,NaN,12.0,NaN,NaN,NaN,2.0,...,71803,0,0,fd7214547e7a6c8b8153a5a26cb57fa373d2653e,0,0,0,0,0,NaN
1,20200613,AL,24601.0,268651.0,NaN,622.0,2241.0,NaN,672.0,NaN,...,293252,4,39,fbbaf3e55677cc7d1eed550a654b675e362268f7,0,0,0,0,0,NaN
2,20200613,AR,11547.0,175470.0,NaN,203.0,979.0,NaN,NaN,49.0,...,187017,0,18,51267156707bdb86f0a40abe6d5a71bc07b53e10,0,0,0,0,0,NaN
3,20200613,AS,0.0,174.0,NaN,NaN,NaN,NaN,NaN,NaN,...,174,0,0,6de298564b96651ea5ba89430d7ace0132db497f,0,0,0,0,0,NaN
4,20200613,AZ,34458.0,293939.0,NaN,1412.0,3692.0,447.0,NaN,309.0,...,328397,39,70,25524dee11d0758cb69b3a22cd09636269c5dc94,0,0,0,0,0,NaN


In [84]:
# change YYYYMMDD to YYYY-MM-DD and convert to datetime

states_hist['date'] = states_hist['date'].apply(lambda x : str(x)[:4] + "-" + str(x)[4:6] + "-" + str(x)[6:8])
states_hist['date'] = states_hist['date'].apply(lambda x : pd.to_datetime(x) )

In [85]:
# Now set the date as the index

states_hist.set_index('date', inplace=True)
states_hist.sort_index(inplace=True)

In [86]:
states_hist.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-01-22,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,2d109d8ba4c74e86fa7202b7c9f26c7617945c58,0,0,0,0,0,NaN
2020-01-23,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,7ed011fe93129bbc13213e7028e7e1e299d67691,0,0,0,0,0,NaN
2020-01-24,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,488430e6f0d933476ca2120fe0ee2ecf4aac5b94,0,0,0,0,0,NaN
2020-01-25,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,4a8a13ff65ef1f0bb991324d98609c765c5b5849,0,0,0,0,0,NaN
2020-01-26,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,5372bd69fec11b6097c548b149e4e79fca8d21d2,0,0,0,0,0,NaN


In [87]:
# get the latest date so we can pull recent data as needed
ctp_date = states_hist.index.max()
ctp_date

Timestamp('2020-06-13 00:00:00')

In [88]:
# get a list of the top 15 states using current data
top_states = states_curr['positive'].sort_values(ascending=False).nlargest(15).index
top_states

Index(['NY', 'NJ', 'CA', 'IL', 'MA', 'TX', 'PA', 'FL', 'MI', 'MD', 'GA', 'VA',
       'LA', 'CT', 'NC'],
      dtype='object', name='state')

In [89]:
# Now create a pivot tables for positive, negative, pending, hospitalized, death, total

states_positive = pd.pivot_table(states_hist, index='date', columns='state', values='positive')
states_negative = pd.pivot_table(states_hist, index='date', columns='state', values='negative')
states_pending = pd.pivot_table(states_hist, index='date', columns='state', values='pending')
states_hospitalized = pd.pivot_table(states_hist, index='date', columns='state', values='hospitalized')
states_deaths = pd.pivot_table(states_hist, index='date', columns='state', values='death')
states_tests = pd.pivot_table(states_hist, index='date', columns='state', values='total')

## Start examining the data - Exploratory Data Analysis (EDA)

In [90]:
# Total Global confirmed cases
jh_live_global['Confirmed'].sum()

7766952

In [91]:
# Total Global deaths
jh_live_global['Deaths'].sum()

429736

In [92]:
# Total US confirmed cases
jh_live_global[jh_live_global["Country_Region"]=='US']['Confirmed'].sum()

2074526

In [93]:
# Total US Deaths (per JH dataset)
jh_live_global[jh_live_global["Country_Region"]=='US']['Deaths'].sum()

115436

Make a Pivot Table to summarize the global confirmed cases by Country  
**DONE**  
date_columns = global_confirmed.iloc[:, 4:].columns
confirmed_country = pd.pivot_table(global_confirmed, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
confirmed_country = confirmed_country.transpose()
confirmed_country.index = pd.to_datetime(confirmed_country.index)
confirmed_country = confirmed_country.sort_index()

In [94]:
counties_CA_filter = counties['state']=='California'
counties_CA_cases = pd.pivot_table(counties[counties_CA_filter], index='date', columns='county', values='cases', aggfunc=np.sum)

In [95]:
top_CA_counties = counties_CA_cases.loc[nytdate, :].nlargest(10).index
top_CA_counties

Index(['Los Angeles', 'Riverside', 'San Diego', 'Orange', 'San Bernardino',
       'Alameda', 'Imperial', 'Kern', 'Santa Clara', 'San Francisco'],
      dtype='object', name='county')

In [96]:
counties_MA_filter = counties['state']=='Massachusetts'
counties_MA_cases = pd.pivot_table(counties[counties_MA_filter], index='date', columns='county', values='cases', aggfunc=np.sum)

In [97]:
top_MA_counties = counties_MA_cases.loc[nytdate, :].nlargest(10).index
top_MA_counties

Index(['Middlesex', 'Suffolk', 'Essex', 'Worcester', 'Norfolk', 'Plymouth',
       'Bristol', 'Hampden', 'Barnstable', 'Hampshire'],
      dtype='object', name='county')